In [702]:
#Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Set working directory
working_dir = '/Users/kenleypelzer/Downloads/'

## Define functions

### Functions that manipulate data or check for duplications within a single dataset (hpt or tic)

In [703]:
#Create "Treatment_ID" variable that will be used to define specific cases and will be the basis for merging
def create_var_count_unique(df):
    #Replace null values with string "NA" to make it easier to create the concatenated "Treatment_ID" variable
    df.fillna('NA', inplace=True)
    df['Treatment_ID'] = df['hospital_name'] + '_' + df['hospital_id'] + '_' + df['payer_name'] + '_' + df['plan_name'] + '_' + df['code_type'] + '_' + df['raw_code'] + '_' + df['setting'] + '_' + df['description'] + '_' + df['modifiers'] + '_' + df['standard_charge_methodology'] + '_' + df['additional_payer_notes'] + '_' + df['additional_generic_notes']
    return df

#Take means of numerical variables for cases where a given 'Treatment_ID' value is associated with multiple values of a numerical variable
#(This function just takes the mean over all cases -- so for most cases, it's a trivial mean over one value)
def take_means(df, var_names_num, var_merge):
    var_list = var_names_num + var_merge
    df_num = df[var_list]
    df_cat = df.drop(columns=var_names_num)
    df_means = df_num.groupby('Treatment_ID').mean()
    means_np = df_means.to_numpy()
    df_cat.drop_duplicates(inplace=True, ignore_index=True)
    df_num_new = pd.DataFrame(data=means_np, columns=var_names_num)
    df_merged = pd.concat([df_cat, df_num_new], axis=1)
    return df_merged

#Find cases within a single dataframe (hpt or tic) where a given Treatment_ID variable has multiple rows and more than one unique value for a particular variable
def find_differences(df, id_var):
    cases_with_variation = []
    var_with_variation = []
    num_vals = []
    all_ids = df[id_var].unique()
    df_cols = df.columns
    for case in all_ids:
        df_lim = df[df[id_var]==case]
        for col_name in df_cols:
            unique_vals = df_lim[col_name].unique()
            if len(unique_vals) != 1:
                cases_with_variation.append(case)
                var_with_variation.append(col_name)
                num_vals.append(len(unique_vals))
    show_diffs = pd.DataFrame(data = {'Treatment_ID': cases_with_variation, 'Varying_Info': var_with_variation, 'Num_Values': num_vals})
    show_diffs_table = show_diffs.pivot(columns='Varying_Info', index='Treatment_ID', values='Num_Values')
    show_diffs_table.fillna('No_Variation', inplace=True)
    return show_diffs_table

### Functions that look for differences BETWEEN hpt and tic

In [704]:

#This function checks whether the summary of each categorical variable (number of cases per category for categorical variable) differ between the hpt and tic files.
def compare_dfs_cat(hpt, tic, vars_to_compare):
    for var_name in vars_to_compare:
        df_hpt = pd.DataFrame(hpt[var_name].value_counts())
        df_tic = pd.DataFrame(tic[var_name].value_counts())
        df_merged = pd.merge(how='outer', left=df_hpt, right=df_tic, left_index=True, right_index=True, suffixes=['_hpt', '_tic'])
        name_hpt = var_name + '_hpt'
        name_tic = var_name + '_tic'
        df_merged['difference'] = np.abs(df_merged[name_hpt] - df_merged[name_tic])
        if df_merged['difference'].max() > 0:
            print('Difference in counts found for variable: \'', var_name, '\', further investigation needed.')
        else:
            print('No difference in counts found for variable \'', var_name, '\'')

#This compares values of numerical variables to find cases where hpt and tic differ
def compare_dfs_num(df1, df2, var_merge, suffixes, vars_to_compare):
    with_diffs_df = pd.DataFrame()
    with_diffs_dict = {}
    for var_name in vars_to_compare:
        df = pd.merge(how='outer', left=df1, right=df2, on=var_merge, suffixes=suffixes)
        name1 = var_name + suffixes[0]
        name2 = var_name + suffixes[1]
        df_comp_diffs = df[(df[name1] != df[name2]) & ((df[name1].isna()==False) | (df[name2].isna()==False))]
        if len(df_comp_diffs) != 0:
            print('Number of differences found for var: ', var_name)
            print(len(df_comp_diffs))
            df_comp_diffs = df_comp_diffs[['Treatment_ID', name1, name2]]
            dict_name = var_name + '_diffs'
            dict_name = df_comp_diffs.copy()
            with_diffs_df = pd.concat([with_diffs_df, df_comp_diffs])
            with_diffs_dict.update({var_name: dict_name})         
    return with_diffs_df, with_diffs_dict

#This compares the MEAN values of those variables (taking means within an individual dataset)
def compare_dfs_mean(df1, df2, var_merge, suffixes, vars_to_compare):
    #Replace nulls (for which we are using the string value NA) with the numerical value 999999) so that the string "NA" doesn't cause the mean operation to fail
    #(The mean operation understands to ignore true null values, but we have replaced the true null with the "NA" string for convenience)
    df1.replace(to_replace='NA', value=999999, inplace=True)
    df2.replace(to_replace='NA', value=999999, inplace=True)
    with_diffs_df_means = pd.DataFrame()
    with_diffs_dict_means = {}
    for var_name in vars_to_compare:
        df1_means = pd.DataFrame(df1.groupby(var_merge)[var_name].mean())
        df2_means = pd.DataFrame(df2.groupby(var_merge)[var_name].mean())
        name1 = var_name + suffixes[0]
        name2 = var_name + suffixes[1]
        df = pd.merge(how='outer', left=df1_means, right=df2_means, left_index=True, right_index=True, suffixes=suffixes)
        df['diff'] = df[name1] - df[name2]  
        df_diff = df[df['diff'] != 0]
        if len(df_diff) != 0:
            df_diff.replace(to_replace=999999, value=np.nan, inplace=True)
            print('Differences found for MEANS for var: ', var_name)
            print(len(df_diff))
            dict_name = var_name + '_diffs'
            dict_name = df_diff.copy()
            with_diffs_df_means = pd.concat([with_diffs_df_means, df_diff])
            with_diffs_dict_means.update({var_name: dict_name})
    return with_diffs_df_means, with_diffs_dict_means

#These looks for differences between equivalent variables after the hpt and tic sources have been merged
def check_for_diffs(df, suffixes, original_cols=original_cols):
    for name in original_cols:
        name1 = name + suffixes[0]
        name2 = name + suffixes[1]
        df = df[df[name1] != df[name2]]
        print('Number of differences for variable: ', name)
        print(len(df))

### Function for processing almost-final output

In [705]:
#This keeps just one of the duplicated columns after we merge hpt and tic 
def keep_one(df, suffixes, original_cols=original_cols):
    list_names = []
    for name in original_cols:
        name1 = name + suffixes[0]
        name2 = name + suffixes[1]
        list_names.append(name1)
        df.rename(columns = {name2 : name}, inplace=True)
    df.drop(columns=list_names, inplace=True)
    return df

### Read in and view tables

In [740]:
#Read raw data into Pandas

hpt_loc = working_dir + 'hpt_extract_20250213.csv'
tic_loc = working_dir + 'tic_extract_20250213.csv'

hpt = pd.read_csv(hpt_loc)
tic = pd.read_csv(tic_loc)

#Save column names to use later
original_cols = hpt.columns

In [741]:
#Check that 999999 (which I like to use for NAs) isn't present in dataframe
value_exists = (hpt == 999999).any().any()
print(value_exists)
value_exists = (tic == 999999).any().any()
print(value_exists)

False
False


In [742]:
#Eyeballing data
hpt.head()

,source_file_name,hospital_id,hospital_name,last_updated_on,hospital_state,license_number,payer_name,plan_name,code_type,raw_code,description,setting,modifiers,standard_charge_gross,standard_charge_discounted_cash,standard_charge_negotiated_dollar,standard_charge_negotiated_percentage,standard_charge_min,standard_charge_max,standard_charge_methodology,additional_payer_notes,additional_generic_notes
0,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Aetna,Medicare,CPT,99283,EMERGENCY DEPT VISIT LOW MDM,outpatient,NaN,NaN,NaN,323.34,NaN,83.78,1009.22,fee schedule,NaN,NaN
1,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,HealthFirst,Commercial Enrollees,CPT,43239,EGD BIOPSY SINGLE/MULTIPLE,outpatient,NaN,NaN,NaN,1037.65,NaN,165.40,3206.34,fee schedule,NaN,NaN
2,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Aetna,Commercial,CPT,43239,UPPER GI ENDOSCOPY BIOPSY,outpatient,NaN,NaN,NaN,1246.73,NaN,1246.73,1394.79,fee schedule,NaN,NaN
3,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Cigna,LocalPlus,CPT,99283,HC EMERGENCY DEPT VISIT LVL 3,outpatient,NaN,3744.0,2433.6,1797.00,NaN,225.00,1797.00,other,NaN,per visit
4,13-1740114_montefiore-medical-center_standardc...,62915ae8-8d64-4e2f-b05f-b18edde57a3d,Montefiore Medical Center,2024-07-01,NY,13-1740114,Oscar,Medicare,CPT,43239,EGD BIOPSY SINGLE/MULTIPLE,outpatient,NaN,NaN,NaN,1037.65,NaN,141.77,1815.10,fee schedule,NaN,NaN


In [743]:
tic.head()

,payer,network_name,network_id,network_year_month,network_region,code,code_type,ein,taxonomy_filtered_npi_list,modifier_list,billing_class,place_of_service_list,negotiation_type,arrangement,rate,cms_baseline_schedule,cms_baseline_rate
0,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,872,MS-DRG,131740114,"1003990763,1023202793,1063525152,1063606739,10...",NaN,institutional,NaN,negotiated,ffs,15902.00,IPPS,6829.75
1,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,99283,CPT,131624096,"1003255670,1245759711,1487026522,1598095267,16...",NaN,professional,11,negotiated,ffs,123.86,PFS_NONFACILITY_1320201,76.89
2,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,43239,CPT,131740114,"1700348620,1700892056,1922539964,1942685292",NaN,professional,11,negotiated,ffs,993.92,PFS_NONFACILITY_1320202,424.76
3,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,872,MS-DRG,133971298,"1245635200,1437523537,1528013695,1528432622,15...",NaN,institutional,NaN,negotiated,ffs,27924.63,IPPS,6829.75
4,unitedhealthcare,choice-plus,592bc118-0dac-4f38-949c-11dc9b3a3879,202501,USA,43239,CPT,131740114,1346697315,NaN,professional,11,negotiated,ffs,849.63,PFS_NONFACILITY_1320203,391.85


### Explore inconsistencies in the individual hpt and tic files

In [710]:
#Implement the functions to create the unique 'Treatment_ID' variable that should capture unique payer/code/hospital combinations
hpt = create_var_count_unique(hpt)

#Implement the function to check whether 'Treatment_ID' is duplicated, and which variables differ in the duplicated rows
show_diffs_hpt = find_differences(hpt, 'Treatment_ID')

#Visualize table of resulting Treatment_IDs and variations in variables
#The numbers show the number of distinct values of that variable
#We see that we have 75 Treatment_ID values that have some conflicting information
show_diffs_hpt

Varying_Info,standard_charge_discounted_cash,standard_charge_gross,standard_charge_max,standard_charge_min,standard_charge_negotiated_dollar
Treatment_ID,,,,,
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_Aetna_ASA_CPT_99283_outpatient_EMERGENCY DEPT VISIT LOW MDM_NA_fee schedule_NA_NA,No_Variation,No_Variation,2.0,2.0,2.0
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_Aetna_Commercial_CPT_43239_both_PR EDG TRANSORAL BIOPSY SINGLE/MULTIPLE_NA_fee schedule_NA_NA,2.0,2.0,No_Variation,No_Variation,No_Variation
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_Aetna_Commercial_CPT_99283_outpatient_EMERGENCY DEPT VISIT LOW MDM_NA_fee schedule_NA_NA,No_Variation,No_Variation,2.0,2.0,2.0
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_Aetna_Medicare_CPT_43239_both_PR EDG TRANSORAL BIOPSY SINGLE/MULTIPLE_NA_fee schedule_NA_NA,2.0,2.0,No_Variation,No_Variation,No_Variation
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_Agewell_Agewell_CPT_43239_both_PR EDG TRANSORAL BIOPSY SINGLE/MULTIPLE_NA_fee schedule_NA_NA,2.0,2.0,No_Variation,No_Variation,No_Variation
...,...,...,...,...,...
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_VNSNY_Choice Medicare_CPT_43239_both_PR EDG TRANSORAL BIOPSY SINGLE/MULTIPLE_NA_fee schedule_NA_NA,2.0,2.0,No_Variation,No_Variation,No_Variation
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_WellCare_Medicaid_CPT_43239_both_PR EDG TRANSORAL BIOPSY SINGLE/MULTIPLE_NA_fee schedule_NA_NA,2.0,2.0,No_Variation,No_Variation,No_Variation
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_WellCare_Medicare_CPT_43239_both_PR EDG TRANSORAL BIOPSY SINGLE/MULTIPLE_NA_fee schedule_NA_NA,2.0,2.0,No_Variation,No_Variation,No_Variation


In [711]:
#Implement the functions to create the unique 'Treatment_ID' variable that should capture unique payer/code/hospital combinations
tic = create_var_count_unique(tic)

#Implement the function to check whether 'Treatment_ID' is duplicated, and which variables differ in the duplicated rows
show_diffs_tic = find_differences(tic, 'Treatment_ID')

#Visualize table of resulting Treatment_IDs and variations in variables
#The numbers show the number of distinct values of that variable
#We see that we have 75 Treatment_ID values that have some conflicting information
show_diffs_tic

Varying_Info,standard_charge_discounted_cash,standard_charge_gross,standard_charge_max,standard_charge_min,standard_charge_negotiated_dollar
Treatment_ID,,,,,
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_Aetna_ASA_CPT_99283_outpatient_EMERGENCY DEPT VISIT LOW MDM_NA_fee schedule_NA_NA,No_Variation,No_Variation,2.0,2.0,2.0
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_Aetna_Commercial_CPT_43239_both_PR EDG TRANSORAL BIOPSY SINGLE/MULTIPLE_NA_fee schedule_NA_NA,2.0,2.0,No_Variation,No_Variation,No_Variation
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_Aetna_Commercial_CPT_99283_outpatient_EMERGENCY DEPT VISIT LOW MDM_NA_fee schedule_NA_NA,No_Variation,No_Variation,2.0,2.0,2.0
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_Aetna_Medicare_CPT_43239_both_PR EDG TRANSORAL BIOPSY SINGLE/MULTIPLE_NA_fee schedule_NA_NA,2.0,2.0,No_Variation,No_Variation,No_Variation
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_Agewell_Agewell_CPT_43239_both_PR EDG TRANSORAL BIOPSY SINGLE/MULTIPLE_NA_fee schedule_NA_NA,2.0,2.0,No_Variation,No_Variation,No_Variation
...,...,...,...,...,...
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_VNSNY_Choice Medicare_CPT_43239_both_PR EDG TRANSORAL BIOPSY SINGLE/MULTIPLE_NA_fee schedule_NA_NA,2.0,2.0,No_Variation,No_Variation,No_Variation
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_WellCare_Medicaid_CPT_43239_both_PR EDG TRANSORAL BIOPSY SINGLE/MULTIPLE_NA_fee schedule_NA_NA,2.0,2.0,No_Variation,No_Variation,No_Variation
Montefiore Medical Center_62915ae8-8d64-4e2f-b05f-b18edde57a3d_WellCare_Medicare_CPT_43239_both_PR EDG TRANSORAL BIOPSY SINGLE/MULTIPLE_NA_fee schedule_NA_NA,2.0,2.0,No_Variation,No_Variation,No_Variation


### Explore inconsistencies BETWEEN hpt and tic

In [712]:
#Implement the function to check whether the summary of each variable (number of cases per category for categorical variable) differ between the hpt and tic files.
#No differences were found.

var_names = ['hospital_name', 'hospital_state', 'license_number', 'payer_name', 'plan_name', 'code_type', 'raw_code', 'description', 'setting']
compare_dfs_cat(hpt=hpt, tic=tic, vars_to_compare=var_names)

No difference in counts found for variable ' hospital_name '
No difference in counts found for variable ' hospital_state '
No difference in counts found for variable ' license_number '
No difference in counts found for variable ' payer_name '
No difference in counts found for variable ' plan_name '
No difference in counts found for variable ' code_type '
No difference in counts found for variable ' raw_code '
No difference in counts found for variable ' description '
No difference in counts found for variable ' setting '


In [713]:
#Check that treatment_IDs with duplications/differences are the same between hpt and tic (they are)
hpt_cases = show_diffs_hpt.index.to_list()
tic_cases = show_diffs_tic.index.to_list()
list(set(hpt_cases) - set(tic_cases)) 

[]

In [714]:
#Create list of treatment_IDs that had variations for hpt/tic
dupes = merge_variations.index.to_list()

In [715]:
#List of numerical variables where we wish to check for differences
var_names_num = ['standard_charge_gross', 'standard_charge_discounted_cash',
       'standard_charge_negotiated_dollar',
       'standard_charge_negotiated_percentage', 'standard_charge_min',
       'standard_charge_max']

#Implement function to check for differences between hpt and tic
with_diffs_df, with_diffs_dict = compare_dfs_num(df1=hpt, df2=tic, var_merge='Treatment_ID', suffixes=['_hpt', '_tic'], vars_to_compare=var_names_num)

Number of differences found for var:  standard_charge_gross
118
Number of differences found for var:  standard_charge_discounted_cash
118
Number of differences found for var:  standard_charge_negotiated_dollar
38
Number of differences found for var:  standard_charge_min
38
Number of differences found for var:  standard_charge_max
38


In [716]:
#Create list of treatment_IDs where some difference was shown BETWEEN hpt and tic
diffs = with_diffs_df['Treatment_ID'].unique()

#Demonstrate that all of those treatment_IDs were included among those where duplicate Treatment_IDs were in the individual hpt/tic table
list(set(dupes) - set(diffs)) 


[]

In [717]:
#Print variables that showed differences between hpt and tic
with_diffs_dict.keys()

dict_keys(['standard_charge_gross', 'standard_charge_discounted_cash', 'standard_charge_negotiated_dollar', 'standard_charge_min', 'standard_charge_max'])

In [718]:
#Visualize example -- note that we see cases with the same values for hpt and tic, but in different orders
#Eyeballing, it seems that if we took the mean over conflicting values WITHIN hpt and tic, the means BETWEEN hpt and tic would not differ
with_diffs_dict['standard_charge_gross']

,Treatment_ID,standard_charge_gross_hpt,standard_charge_gross_tic
7,Montefiore Medical Center_62915ae8-8d64-4e2f-b...,3925.0,1790.0
8,Montefiore Medical Center_62915ae8-8d64-4e2f-b...,1790.0,3925.0
11,Montefiore Medical Center_62915ae8-8d64-4e2f-b...,3925.0,1790.0
12,Montefiore Medical Center_62915ae8-8d64-4e2f-b...,1790.0,3925.0
17,Montefiore Medical Center_62915ae8-8d64-4e2f-b...,3925.0,1790.0
...,...,...,...
1656,Montefiore Medical Center_62915ae8-8d64-4e2f-b...,1790.0,3925.0
1666,Montefiore Medical Center_62915ae8-8d64-4e2f-b...,3925.0,1790.0
1667,Montefiore Medical Center_62915ae8-8d64-4e2f-b...,1790.0,3925.0
1696,Montefiore Medical Center_62915ae8-8d64-4e2f-b...,3925.0,1790.0


In [719]:
#We demonstrate that when means are taken over the differing values WITHIN hpt and tic, no differences between hpt and tic are found
with_diffs_df_means, with_diffs_dict_means = compare_dfs_mean(df1=hpt, df2=tic, var_merge='Treatment_ID', suffixes=['_hpt', '_tic'], vars_to_compare=var_names_num)
print(len(with_diffs_df_means))

0


### Process and create new dataframe

In [720]:
#Create new dataframes with means taken over conflicting numerical variables
hpt_new = take_means(df=hpt, var_names_num=var_names_num, var_merge=['Treatment_ID'])
tic_new = take_means(df=hpt, var_names_num=var_names_num, var_merge=['Treatment_ID'])

In [721]:
#Merge new hpt and tic dataframes
new_merge = pd.merge(how='outer', left=hpt_new, right=tic_new, on='Treatment_ID', suffixes = ['_hpt', '_tic'])

In [722]:
#Check whether there are differences between hpt and tic variables
check_for_diffs(new_merge, suffixes=['_hpt', '_tic'])

Number of differences for variable:  source_file_name
0
Number of differences for variable:  hospital_id
0
Number of differences for variable:  hospital_name
0
Number of differences for variable:  last_updated_on
0
Number of differences for variable:  hospital_state
0
Number of differences for variable:  license_number
0
Number of differences for variable:  payer_name
0
Number of differences for variable:  plan_name
0
Number of differences for variable:  code_type
0
Number of differences for variable:  raw_code
0
Number of differences for variable:  description
0
Number of differences for variable:  setting
0
Number of differences for variable:  modifiers
0
Number of differences for variable:  standard_charge_gross
0
Number of differences for variable:  standard_charge_discounted_cash
0
Number of differences for variable:  standard_charge_negotiated_dollar
0
Number of differences for variable:  standard_charge_negotiated_percentage
0
Number of differences for variable:  standard_charge

In [723]:
#Select variables to keep (doesn't matter whether we choose hpt or tic version now that we have checked that they are all available)
new_merge = keep_one(new_merge, suffixes=['_hpt', '_tic'])
new_merge.reset_index(drop=True, inplace=True)

In [724]:
#Add note to cases where conflicting numerical variables were averaged, so that these can be investigated later if necessary/time permitting
new_merge['Notes'] = np.nan
new_merge.loc[new_merge['Treatment_ID'].isin(dupes),'Notes'] = 'Took mean over inconsistent values for charges'

In [725]:
#Drop Treatment_ID, a fictional variable that was just for code
new_merge.drop(columns = ['Treatment_ID'], inplace=True)

#Change any 999999 values back to NA
new_merge.replace(to_replace=999999, value=np.nan, inplace=True)


In [726]:
hpt['raw_code'].value_counts()

43239         1526
99283         1046
MS-DRG 872     290
872             88
Name: raw_code, dtype: int64

In [727]:
hpt['standard_charge_methodology'].value_counts()

Case Rate                          1316
percent of total billed charges     613
fee schedule                        322
Fee Schedule                        224
999999                              214
case rate                            81
Case rate                            80
Percent of total billed charges      51
other                                34
Other                                 8
5228                                  2
per diem                              1
32829.5                               1
11671                                 1
93157.15                              1
17412.69                              1
Name: standard_charge_methodology, dtype: int64

In [728]:
hpt['code_type'].value_counts()

CPT       2162
LOCAL      410
MS-DRG     378
Name: code_type, dtype: int64

In [729]:
cond_type_merge = new_merge['code_type']=='CPT'
new_merge = new_merge[cond_type_merge]
cond_type_setting = new_merge['setting']=='both'
new_merge = new_merge[cond_type_setting]
cond_method = new_merge['standard_charge_methodology']=='Case Rate'
new_merge = new_merge[cond_method]

print(len(new_merge))

new_merge.isna().sum()

1078


source_file_name                            0
hospital_id                                 0
hospital_name                               0
last_updated_on                             0
hospital_state                              0
license_number                              0
payer_name                                  0
plan_name                                   0
code_type                                   0
raw_code                                    0
description                                 0
setting                                     0
modifiers                                1078
standard_charge_methodology                 0
additional_payer_notes                      0
additional_generic_notes                 1078
standard_charge_gross                     333
standard_charge_discounted_cash           326
standard_charge_negotiated_dollar         177
standard_charge_negotiated_percentage     796
standard_charge_min                       938
standard_charge_max               

In [730]:
#Drop variables that are either redundant or have all null values
new_merge.drop(columns = ['source_file_name', 'license_number', 'hospital_id', 'last_updated_on', 'hospital_state', 'modifiers', 'additional_generic_notes'], inplace=True)

In [734]:
new_merge.reset_index(drop=True, inplace=True)

In [735]:
#Print to csv
new_merge.to_csv('TakeHome.csv')

In [733]:
print(hpt[condition_hpt_dupes]['standard_charge_methodology'].value_counts())
print(hpt[~condition_hpt_dupes]['standard_charge_methodology'].value_counts())

fee schedule    152
Name: standard_charge_methodology, dtype: int64
Case Rate                          1316
percent of total billed charges     613
Fee Schedule                        224
999999                              214
fee schedule                        170
case rate                            81
Case rate                            80
Percent of total billed charges      51
other                                34
Other                                 8
5228                                  2
per diem                              1
32829.5                               1
11671                                 1
93157.15                              1
17412.69                              1
Name: standard_charge_methodology, dtype: int64
